# DDSM classification

In [1]:
from __future__ import print_function

import os
import torch
import torch.optim as optim
from torch.autograd import Variable

# Internal dependency
import classification as ddsm_classify
from classification import DDSMDataset
from classification import MyResNet

## Set up training variables

In [2]:
# Change these depending on what machine this notebook is running on
# data_dir = "/Users/yairschiff/Development/PycharmProjects/ComputerVision/Project/data/"
data_dir = "/scratch/jtb470/DDSM/data"
# model_res_dir = "/Users/yairschiff/Development/PycharmProjects/ComputerVision/Project/model_results_stage2/"
model_res_dir = "/scratch/yzs208/CV_Project/model_results_s2_nb"
batch_size = 2
epochs = 20
lr = 0.01
# checkpoint = "/Users/yairschiff/Development/PycharmProjects/ComputerVision/Project/model_results/model_results20.pth"
checkpoint = "/scratch/yzs208/CV_Project/model_results_v2/model_results20.pth"
train_heads = checkpoint == ""
log_interval = 50
torch.manual_seed(1)

## Set device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != torch.device("cpu"):
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
print("Device: {}".format(device))

Device: cuda:0


## Load data

In [4]:
train_loader = torch.utils.data.DataLoader(DDSMDataset(data_dir, dataset="train", exclude_brightened=True),
                                           batch_size=batch_size, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(DDSMDataset(data_dir, dataset="val", exclude_brightened=True),
                                         batch_size=batch_size, shuffle=True, num_workers=1)

## Load model

In [5]:
model = MyResNet("resnet18", 3, only_train_heads=train_heads)
if checkpoint != "":
    state_dict = torch.load(checkpoint) if torch.cuda.is_available() else torch.load(checkpoint, map_location='cpu')
    model.load_state_dict(state_dict)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

## Run training and validation

In [6]:
if not os.path.isdir(model_res_dir):
    print(model_res_dir + " not found: making directory for results")
    os.mkdir(model_res_dir)
for epoch in range(1, epochs + 1):
    ddsm_classify.train(model, train_loader, optimizer, device, epoch, log_interval)
    ddsm_classify.validation(model, val_loader, device)
    model_file = os.path.join(model_res_dir, "model_stage" +
                              str(1 if train_heads else 2) + "_" + str(epoch) + ".pth")
    torch.save(model.state_dict(), model_file)
    print("\nSaved model to " + model_file + ".")

Training epoch 1...
Train Epoch: 1 [100/6659 (2%)]	Loss: 304.887207
Train Epoch: 1 [200/6659 (3%)]	Loss: 0.000000
Train Epoch: 1 [300/6659 (5%)]	Loss: 0.000000
Train Epoch: 1 [400/6659 (6%)]	Loss: 83.091812
Train Epoch: 1 [500/6659 (8%)]	Loss: 130.121399
Train Epoch: 1 [600/6659 (9%)]	Loss: 389.902405
Train Epoch: 1 [700/6659 (11%)]	Loss: 15.930120
Train Epoch: 1 [800/6659 (12%)]	Loss: 9.845179
Train Epoch: 1 [900/6659 (14%)]	Loss: 21.386019
Train Epoch: 1 [1000/6659 (15%)]	Loss: 64.317009
Train Epoch: 1 [1100/6659 (17%)]	Loss: 7.687445
Train Epoch: 1 [1200/6659 (18%)]	Loss: 60.810493
Train Epoch: 1 [1300/6659 (20%)]	Loss: 9.447364
Train Epoch: 1 [1400/6659 (21%)]	Loss: 2.458696
Train Epoch: 1 [1500/6659 (23%)]	Loss: 11.876880
Train Epoch: 1 [1600/6659 (24%)]	Loss: 0.597017
Train Epoch: 1 [1700/6659 (26%)]	Loss: 0.547805
Train Epoch: 1 [1800/6659 (27%)]	Loss: 6.281961
Train Epoch: 1 [1900/6659 (29%)]	Loss: 0.046251
Train Epoch: 1 [2000/6659 (30%)]	Loss: 0.245591
Train Epoch: 1 [2100/66

Train Epoch: 3 [3300/6659 (50%)]	Loss: 0.764695
Train Epoch: 3 [3400/6659 (51%)]	Loss: 0.786069
Train Epoch: 3 [3500/6659 (53%)]	Loss: 1.386921
Train Epoch: 3 [3600/6659 (54%)]	Loss: 0.857590
Train Epoch: 3 [3700/6659 (56%)]	Loss: 0.816765
Train Epoch: 3 [3800/6659 (57%)]	Loss: 1.207274
Train Epoch: 3 [3900/6659 (59%)]	Loss: 0.461796
Train Epoch: 3 [4000/6659 (60%)]	Loss: 0.887947
Train Epoch: 3 [4100/6659 (62%)]	Loss: 0.943437
Train Epoch: 3 [4200/6659 (63%)]	Loss: 0.808438
Train Epoch: 3 [4300/6659 (65%)]	Loss: 0.966520
Train Epoch: 3 [4400/6659 (66%)]	Loss: 0.891791
Train Epoch: 3 [4500/6659 (68%)]	Loss: 0.764720
Train Epoch: 3 [4600/6659 (69%)]	Loss: 1.008672
Train Epoch: 3 [4700/6659 (71%)]	Loss: 1.004657
Train Epoch: 3 [4800/6659 (72%)]	Loss: 0.706748
Train Epoch: 3 [4900/6659 (74%)]	Loss: 1.445769
Train Epoch: 3 [5000/6659 (75%)]	Loss: 0.645702
Train Epoch: 3 [5100/6659 (77%)]	Loss: 0.863510
Train Epoch: 3 [5200/6659 (78%)]	Loss: 0.792445
Train Epoch: 3 [5300/6659 (80%)]	Loss: 0

Train Epoch: 5 [6600/6659 (99%)]	Loss: 0.938854

Validation set: Average loss: 1.0017, Accuracy: 292/738 (39%)


Saved model to /scratch/yzs208/CV_Project/model_results_s2_nb/model_stage2_5.pth.
Training epoch 6...
Train Epoch: 6 [100/6659 (2%)]	Loss: 0.882146
Train Epoch: 6 [200/6659 (3%)]	Loss: 0.911983
Train Epoch: 6 [300/6659 (5%)]	Loss: 0.816110
Train Epoch: 6 [400/6659 (6%)]	Loss: 0.736809
Train Epoch: 6 [500/6659 (8%)]	Loss: 0.824035
Train Epoch: 6 [600/6659 (9%)]	Loss: 0.807546
Train Epoch: 6 [700/6659 (11%)]	Loss: 1.471943
Train Epoch: 6 [800/6659 (12%)]	Loss: 0.759265
Train Epoch: 6 [900/6659 (14%)]	Loss: 0.681792
Train Epoch: 6 [1000/6659 (15%)]	Loss: 1.619375
Train Epoch: 6 [1100/6659 (17%)]	Loss: 0.611941
Train Epoch: 6 [1200/6659 (18%)]	Loss: 1.403646
Train Epoch: 6 [1300/6659 (20%)]	Loss: 0.700568
Train Epoch: 6 [1400/6659 (21%)]	Loss: 0.835276
Train Epoch: 6 [1500/6659 (23%)]	Loss: 1.431264
Train Epoch: 6 [1600/6659 (24%)]	Loss: 0.716648
Train Epoch: 6 [1700/6659 (26%)]

Train Epoch: 8 [3000/6659 (45%)]	Loss: 0.823887
Train Epoch: 8 [3100/6659 (47%)]	Loss: 0.824867
Train Epoch: 8 [3200/6659 (48%)]	Loss: 0.827260
Train Epoch: 8 [3300/6659 (50%)]	Loss: 0.818840
Train Epoch: 8 [3400/6659 (51%)]	Loss: 0.822116
Train Epoch: 8 [3500/6659 (53%)]	Loss: 0.832187
Train Epoch: 8 [3600/6659 (54%)]	Loss: 0.811922
Train Epoch: 8 [3700/6659 (56%)]	Loss: 0.825037
Train Epoch: 8 [3800/6659 (57%)]	Loss: 0.824608
Train Epoch: 8 [3900/6659 (59%)]	Loss: 1.590129
Train Epoch: 8 [4000/6659 (60%)]	Loss: 0.842055
Train Epoch: 8 [4100/6659 (62%)]	Loss: 0.834475
Train Epoch: 8 [4200/6659 (63%)]	Loss: 1.611603
Train Epoch: 8 [4300/6659 (65%)]	Loss: 0.729200
Train Epoch: 8 [4400/6659 (66%)]	Loss: 1.505675
Train Epoch: 8 [4500/6659 (68%)]	Loss: 1.505925
Train Epoch: 8 [4600/6659 (69%)]	Loss: 0.669071
Train Epoch: 8 [4700/6659 (71%)]	Loss: 1.007931
Train Epoch: 8 [4800/6659 (72%)]	Loss: 0.835260
Train Epoch: 8 [4900/6659 (74%)]	Loss: 0.878047
Train Epoch: 8 [5000/6659 (75%)]	Loss: 0

Train Epoch: 10 [6200/6659 (93%)]	Loss: 0.742036
Train Epoch: 10 [6300/6659 (95%)]	Loss: 2.050083
Train Epoch: 10 [6400/6659 (96%)]	Loss: 0.773799
Train Epoch: 10 [6500/6659 (98%)]	Loss: 0.881872
Train Epoch: 10 [6600/6659 (99%)]	Loss: 0.807011

Validation set: Average loss: 0.9891, Accuracy: 351/738 (47%)


Saved model to /scratch/yzs208/CV_Project/model_results_s2_nb/model_stage2_10.pth.
Training epoch 11...
Train Epoch: 11 [100/6659 (2%)]	Loss: 0.826734
Train Epoch: 11 [200/6659 (3%)]	Loss: 1.427381
Train Epoch: 11 [300/6659 (5%)]	Loss: 0.791350
Train Epoch: 11 [400/6659 (6%)]	Loss: 0.825138
Train Epoch: 11 [500/6659 (8%)]	Loss: 0.862282
Train Epoch: 11 [600/6659 (9%)]	Loss: 1.461446
Train Epoch: 11 [700/6659 (11%)]	Loss: 0.823815
Train Epoch: 11 [800/6659 (12%)]	Loss: 0.820986
Train Epoch: 11 [900/6659 (14%)]	Loss: 0.781015
Train Epoch: 11 [1000/6659 (15%)]	Loss: 0.790910
Train Epoch: 11 [1100/6659 (17%)]	Loss: 0.868805
Train Epoch: 11 [1200/6659 (18%)]	Loss: 0.836382
Train Epoch: 

Train Epoch: 13 [2200/6659 (33%)]	Loss: 0.689395
Train Epoch: 13 [2300/6659 (35%)]	Loss: 1.395548
Train Epoch: 13 [2400/6659 (36%)]	Loss: 0.848289
Train Epoch: 13 [2500/6659 (38%)]	Loss: 1.342399
Train Epoch: 13 [2600/6659 (39%)]	Loss: 0.662512
Train Epoch: 13 [2700/6659 (41%)]	Loss: 0.807098
Train Epoch: 13 [2800/6659 (42%)]	Loss: 0.831592
Train Epoch: 13 [2900/6659 (44%)]	Loss: 0.837894
Train Epoch: 13 [3000/6659 (45%)]	Loss: 0.854841
Train Epoch: 13 [3100/6659 (47%)]	Loss: 0.842074
Train Epoch: 13 [3200/6659 (48%)]	Loss: 0.841716
Train Epoch: 13 [3300/6659 (50%)]	Loss: 0.959631
Train Epoch: 13 [3400/6659 (51%)]	Loss: 1.402897
Train Epoch: 13 [3500/6659 (53%)]	Loss: 0.831953
Train Epoch: 13 [3600/6659 (54%)]	Loss: 0.744295
Train Epoch: 13 [3700/6659 (56%)]	Loss: 0.826846
Train Epoch: 13 [3800/6659 (57%)]	Loss: 1.008841
Train Epoch: 13 [3900/6659 (59%)]	Loss: 0.681476
Train Epoch: 13 [4000/6659 (60%)]	Loss: 0.747413
Train Epoch: 13 [4100/6659 (62%)]	Loss: 1.011353
Train Epoch: 13 [420

KeyboardInterrupt: 